In [6]:
import pandas as pd
import json
from google.cloud import bigquery
import utils as u

In [2]:
queries = u.get_queries('queries')

In [3]:
bq_client = bigquery.Client()

In [4]:
data = u.get_data(queries, 'all_gmaps_responses.sql', bq_client)

In [8]:
data['response'] = data['response'].apply(json.loads)

In [9]:
data.head()

,post_code,lat,lng,radius,response
0,2018,59.9375,11.0216,500,"{'html_attributions': [], 'next_page_token': '..."
1,1101,59.8732,10.8107,500,"{'html_attributions': [], 'next_page_token': '..."
2,0514,59.9393,10.7960,500,"{'html_attributions': [], 'next_page_token': '..."
3,0369,59.9321,10.7068,500,"{'html_attributions': [], 'next_page_token': '..."
4,0380,59.9321,10.6552,500,"{'html_attributions': [], 'next_page_token': '..."


In [14]:
data.loc[0, 'response']['results'][0]

{'geometry': {'location': {'lat': 59.93799919999999, 'lng': 11.021436},
  'viewport': {'northeast': {'lat': 59.93938763029151,
    'lng': 11.0228280302915},
   'southwest': {'lat': 59.93668966970849, 'lng': 11.02013006970849}}},
 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png',
 'id': '66be426ab3fc9493647e77a4ffe1c2bd05be68ae',
 'name': 'Løvenstad frisør Nina Kjær',
 'place_id': 'ChIJ8UOD5lJ7QUYRUdZcUVn5QP8',
 'plus_code': {'compound_code': 'W2QC+5H Løvenstad, Norway',
  'global_code': '9FFHW2QC+5H'},
 'rating': 4.5,
 'reference': 'ChIJ8UOD5lJ7QUYRUdZcUVn5QP8',
 'scope': 'GOOGLE',
 'types': ['hair_care', 'point_of_interest', 'establishment'],
 'user_ratings_total': 2,
 'vicinity': 'Løvenstadtorget 18, Løvenstad'}

In [15]:
def get_lat(place):
    return place.get('geometry').get('location').get('lat')

def get_lng(place):
    return place.get('geometry').get('location').get('lng')

def get_id(place):
    return place.get('id')

In [28]:
result = []
columns = ['post_code', 'radius', 'place_name', 'place_type', 'place_attribute', 'attribute_value']

for radius in data.radius.unique():
    for post_code, response in zip(data.post_code, data.response):
        for place in response['results']:
            place_name = place['name']
            for place_type in place['types']:
                for attr_f, attr_n in zip([get_lat, get_lng], ['lat', 'lng']):
                    try:
                        value = attr_f(place)
                    except AttributeError:
                        value = None
                    result.append([post_code, radius, place_name, place_type, attr_n, value])

In [29]:
post_code_places = pd.DataFrame(result, columns=columns)

In [30]:
post_code_places.head()

,post_code,radius,place_name,place_type,place_attribute,attribute_value
0,2018,500,Løvenstad frisør Nina Kjær,hair_care,lat,59.937999
1,2018,500,Løvenstad frisør Nina Kjær,hair_care,lng,11.021436
2,2018,500,Løvenstad frisør Nina Kjær,point_of_interest,lat,59.937999
3,2018,500,Løvenstad frisør Nina Kjær,point_of_interest,lng,11.021436
4,2018,500,Løvenstad frisør Nina Kjær,establishment,lat,59.937999


In [31]:
post_code_places.to_gbq(destination_table='neighborhood_data.post_code_places',
                        project_id='hde-test-clean',
                        if_exists='append')